<a id='top'></a>
# PDRs4All: Reducing NIRCam Imaging Data
---
**Author**: Amélie Canin (amelie.canin@irap.omp.eu) 

**Latest Update**: 15 juin 2022

This notebook follows JWebbinar example from STScI's available on the [Git](https://github.com/spacetelescope/jwebbinar_prep) and on the [website](https://www.stsci.edu/jwst/science-execution/jwebbinars) to reduce NIRCam imaging data using the STScI data reduction pipeline.

<div class="alert alert-block alert-info">
    <h3><u><b>Notebook Goals</b></u></h3>
    <ul>Take a PDRs4All NIRCam imaging simulated with MIRAGE through all three stages of the JWST Calibration Pipeline. We will:</ul>
    <ul>    
      <li>call the pipeline on a sigle image; </li>
      <li>reduce image for a short and a long wavelenght filter. </li> 
    </ul>
</div>

## Introduction

<img align="left" width=15% src="pdrs4all_logo.png">

### The PDRs4All: Radiative feeback from massive stars

PDRs4All is a program of imaging and spectroscopy in the Orion Bar to understand the impact of massive stars. 
Massive stars disrupt their natal molecular cloud material by dissociating molecules, ionizing atoms and molecules, and heating the gas and dust. These processes drive the evolution of interstellar matter in our Galaxy and throughout the Universe from the era of vig- orous star formation at redshifts of 1-3, to the present day. Much of this interaction occurs in Photo-Dissociation Regions where far-ultraviolet photons of these stars create a largely neutral, but warm region of gas and dust.

PDRs4All has 12 NIRCam imaging filters in the same field of view to observe different Polycyclic Aromatic Hydrocarbons on the Orion Bar in the field of view on the [figure below](#fov). More detailled of the PDRs4All are available in the [Berné, Habart, Peeters et al. 2022](https://iopscience.iop.org/article/10.1088/1538-3873/ac604c) and on the [website](https://pdrs4all.org).

<a id='fov'></a>
<img width=30% src="spitzer_fov.jpeg">

In this notebook, we will demonstrate how to reduce NIRCam imaging observations with the Data Reduction Pipeline from the STScI. There is 3 ways to use the pipeline, in this notebook we will only see one. To have more information on the others way see [documentation](#doc) and former [Webbinars](https://www.stsci.edu/jwst/science-execution/jwebbinars).

### Simulated Data

We have simulated PDRs4All observation using [MIRAGE (Multi-Instrument RAmp GEnerator)](https://mirage-data-simulator.readthedocs.io/en/latest/). The simulator use two images as background for the filter F140M, the Hubble Space Telescope obtained by [Robberto et al. 2020](https://iopscience.iop.org/article/10.3847/1538-4357/ab911e) at 1.4 μm with the Wide Field Camera 3 (WFC3) and for the filter F335M the Spitzer-IRAC obtained by [Megeath et al. 2015](https://iopscience.iop.org/article/10.3847/0004-6256/151/1/5) at 3.6 μm.

The exemple for NIRCam imaging simulation is available in the precedent notebook.

For more information on the simulation see [Canin et al. 2021](https://arxiv.org/abs/2112.03106).

<a id='doc'></a>
## Ressources and Documentation

There are several different places to find information on installing and running the pipeline. This notebook will provide examples of running the pipeline on a handful of images, but will not demonstrate all options and features. Please see the following links for more in-depth instructions and documentation.

* [High-level description of all pipeline stages and steps](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html) from the `jwst` software documentation pages.

* JWST Documentation (JDox) for each pipeline stage, including a short summary of what each step does:

  * [JDox page for the Stage 1 pipeline](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview/stages-of-jwst-data-processing/calwebb_detector1) 

  * [JDox page for the Stage 2 pipeline](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview/stages-of-jwst-data-processing/calwebb_image2)
  
  * [JDox page for the Stage 3 pipeline](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview/stages-of-jwst-data-processing/calwebb_image3)

* [`jwst` package documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html) including how to run the pipeline, input/output files, etc.

* [`jwst` package GitHub repository](https://github.com/spacetelescope/jwst/blob/master/README.md), including installation instructions

* [Help Desk](https://stsci.service-now.com/jwst?id=sc_cat_item&sys_id=27a8af2fdbf2220033b55dd5ce9619cd&sysparm_category=e15706fc0a0a0aa7007fc21e1ab70c2f): If you have any questions or problems regarding the pipeline, submit a ticket to the Help Desk

### Installation

<div class="alert alert-block alert-info">
    Before running this notebook, you will have to first install the <code>jwst</code> package.
    
**NOTE:** The `jwst` package requries Python 3.8+ <br><br>
    
The best way to install the pipeline is via `pip`. The steps below is the way to create a new conda environment, activate it, and then install the latest released version of the pipeline. The name of your environment has impact. In the lines below, replace `<env_name>` with your chosen environment name.

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install jwst`

For more detailed instructions on the various ways to install the package, including installing more recent development versions of the pipeline, see the [installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md) on GitHub.
    
</div>

### Reference files

[Calibration reference files](https://jwst-docs.stsci.edu/data-processing-and-calibration-files/calibration-reference-files) are a collection of FITS and ASDF files that are used to remove instrumental signatures and calibrate JWST data. For example, the dark current reference file contains a multiaccum ramp of dark current signal to be subtracted from the data during the dark current subtraction step. 

When running a pipeline or pipeline step, the pipeline will automatically look for any required reference files in a pre-defined local directory. If the required reference files are not present, they will automatically be downloaded from the Calibration Reference Data System (CRDS) at STScI.

<div class="alert alert-block alert-info">
    
You will have to specify a local directory in which to store reference files, along with the server to use to download the reference files from CRDS. To accomplish this, there are two environment variables that should be set prior to calling the pipeline. These are the `CRDS_PATH` and `CRDS_SERVER_URL` variables. In the example below, reference files will be downloaded to the "crds_cache" directory under the home directory.

>`$ export CRDS_PATH=$HOME/crds_cache`<br>
>`$ export CRDS_SERVER_URL=https://jwst-crds.stsci.edu`<br>
OR:<br>
`os.environ["CRDS_PATH"] = "/user/myself/crds_cache"`<br>
`os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"`<br>

The first time you run the pipeline, the CRDS server should download all of the context and reference files that are needed. 
</div>

<div class="alert alert-block alert-warning">

To get the latest version of the reference files, use `https://jwst-crds.stsci.edu`.

More informations on the [documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html#crds).
    
</div>

In [ ]:
import os

# Make sure to replace with the path to your CRDS cache and mirage data directories
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds-pub.stsci.edu"
os.environ["CRDS_PATH"] = "/path/to/my/files/"
os.environ["CRDS_CONTEXT"] = "jwst_0862.pmap"

## Import

In [ ]:
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Image3Pipeline
import jwst.associations
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

from glob import glob
import astropy.io.fits as fits

Set up matplotlib for plotting

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Use this version for non-interactive plots (easier scrolling of the notebook)
#%matplotlib inline

# Use this version if you want interactive plots
#%matplotlib notebook
%matplotlib inline

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
#%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

# You may want to change the following configurations to customize 
# figure sizes and resolutions
rcParams['figure.figsize'] = [11,8]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

Check which version of the pipeline you are running

In [ ]:
import jwst
print(jwst.__version__)

Function to show the different images

In [ ]:
def show(array,title,unit,min=0,max=1000):
    plt.figure(figsize=(10,10))
    plt.imshow(array,clim=(min,max),origin='lower')
    plt.title(title)
    plt.colorbar().set_label(unit)

## Pipeline for Short Wavelength

The JWST Data Reduction Pipeline is a Python package developed by the STScI. This software allows us to produce calibrated and reduced data from raw data taken by the JWST. From the raw data to the final data, the pipeline is composed of three stages ([figure](#pipeline)). In the following sections, we describe the three stages and code for processing the uncalibrated data to obtain Stage 3 data, applied to the raw images created in the previous notebook.

<a id='pipeline'></a>
<img src="pipeline-overview.png">

In this section, we will run the pipeline on the observations with the filter F140M, corresponding to short wavelengths. For short wavelengths, the used detectors are A1, A2, A3, A4, B1, B2, B3 and B4 but in our simulated data, we only have the observations for the module B.

### calwebb_detector1: Stage 1 Detector Processing

**Description**

This stage is the first one for all of the instruments. It corrects a part of instrument signatures in particu- lar by removing the readout pattern (the stripes). It is called`CALWEBB DETECTOR1` or `Detector1Pipeline` in the Python pipeline. The output of MIRAGE can be used here if during the simulation, datatype is set to raw. This stage only takes one file at a time and returns uncalibrated slope images in units of ADU/sec.

**Inputs**

* A raw exposure (`*_uncal.fits`) containing the 4-dimensional raw data from all detector readouts: (nintegrations x ngroups x ncols x nrows)


**Outputs**

* A 3D countrate image (`*_rateints.fits`) containing the results of each integration
* A 2D countrate image (`*_rate.fits`) corresponding of the average over the exposure's integrations

If there is only one integrate, `*_rateints.fits` and `*_rate.fits` are the same.


Download data here:
- [jw01288001001_01101_00021_nrcb1_uncal.fits](https://stsci.box.com/s/7vmwyio6p07pm4wbzaysiisk3s14uzq7)
- [jw01288001001_01101_00022_nrcb1_uncal.fits](https://stsci.box.com/s/vv9fphlxgljdgn5huf2d49j4y1970t7w)
- [jw01288001001_01101_00023_nrcb1_uncal.fits](https://stsci.box.com/s/v9dtwauy85fl8tdjppd5c5me2n4ozyw2)
- [jw01288001001_01101_00024_nrcb1_uncal.fits](https://stsci.box.com/s/nizzf03dwtoik6guq8o3crk8hyfl9k4m)

More data will eventually be available on the PDRs4All webpage.

In [ ]:
# Path containing the raw data
path_stage0 = './path/to/downloaded/data/'

Function to run the stage 1 with the options we want

<div class="alert alert-block alert-info">
    
In each stage, the option `save_results` is set to `True` to save the results on the laptop at the location `output_dir`.
    
</div>

In [ ]:
# Function to run the stage 1 of the pipeline
def rundet1(filename, output_dir):
    # Instantiate the pipeline
    det1 = Detector1Pipeline()
    # Specify where the output should go
    det1.output_dir = output_dir
    # Save the final resulting _rate.fits files
    det1.save_results = True
    # Prevents saturation
    det1.ramp_fit.suppress_one_group = False
    # Run the pipeline on an input list of files
    det1(filename)

In [ ]:
# Raw data
uncal_files = glob(os.path.join(path_stage0, '*b1_uncal.fits'))

In [ ]:
# Create a folder to store stage 1 products
path_stage1 = path_stage0.replace('stage0', 'stage1')
if not os.path.exists(path_stage1):
    os.makedirs(path_stage1)

In [ ]:
# Run the stage 1 on all the raw data
for f in uncal_files:
    rundet1(f,path_stage1)

Plot the result of the rate file for the detector B1 and one dither

In [ ]:
rate_data = fits.getdata(os.path.join(path_stage1, 'jw01288001001_01101_00021_nrcb1_rate.fits'))

In [ ]:
show(rate_data,'Rate data', 'ADU/sec', max=40)

### calwebb_image2: Stage 2 Imaging Processing

**Description**

This stage corrects other instrumental signatures and calibrates the exposures. This step is performed using the `CALWEBB IMAGE` or `Image2Pipeline` in the Pipeline. This stage returns calibrated but unrectified slope images in units of MJy/sr.

**Inputs**

* A 2D countrate image (`*_rate.fits`) corresponding of the average over the exposure's integrations


**Outputs**

* A 2D calibrated, but unrectified, exposure (`*_cal.fits`)
* A 2D resampled, or rectified, image (`*_i2d.fits`)

The `*_i2d.fits` should not be used for science case because it did not pass through stage 3. At this stage, it can be used only for quick-look.

Function to run the stage 2 with the options we want

In [ ]:
# Function to run the stage 1 of the pipeline
def runimg2(filename, output_dir):
    # Instantiate the pipeline
    img2 = Image2Pipeline()
    # Specify where the output should go
    img2.output_dir = output_dir
    # Save the final resulting _rate.fits files
    img2.save_results = True
    # Run the pipeline on an input list of files
    img2(filename)

In [ ]:
# Rate data
rate_files = glob(os.path.join(path_stage1, '*rate.fits'))

# Create a folder to store stage 2 products
path_stage2 = path_stage0.replace('stage0', 'stage2')

if not os.path.exists(path_stage2):
    os.makedirs(path_stage2)

In [ ]:
# Run the stage 2 on all the rate data
for f in rate_files:
    runimg2(f,path_stage2)

Plot the result of the calibrated file for the detector B1 and one dither

In [ ]:
cal_data = fits.getdata(os.path.join(path_stage2, 'jw01288001001_01101_00021_nrcb1_cal.fits'))

In [ ]:
show(cal_data, 'Calibrated data', 'MJy/sr', max = 200)

### calwebb_image3: Stage 3 Imaging Processing

**Description**

This stage combines multiple exposures from dithers or mosaics and rectifies the exposures to produce one unique mosaic, the final product. In this stage, we use the imaging mode called `CALWEBB IMAGE3` or `Image3Pipeline` in the Pipeline. To combine multiple exposures, an association (ASN) file is created which contains the images to be combined. Any combination of detectors and dithers is possible. This returns the final mosaic image, rectified, in units of MJy/sr.

**Inputs**

* A 2D calibrated, but unrectified, exposures (`*_cal.fits`) combined in an ASN file


**Outputs**

* A 2D resampled and combined image (`*_i2d.fits`) containing the combined and rectified association of exposures
* A 2D cosmic-ray flagged exposure (`*_crf.fits`) if `outlier_step == True`
* A source catalog (`*_cat.ecsv`) saved in ASCII
* A 2D segmentation map (`*_segm.fits`)


<div class="alert alert-block alert-info">
    
The stage 3 take in input an association (ASN) file to combined multiple exposures. The paths of the exposures are writen in a `JSON` file. 

We use the function `jwst.associations.asn_from_list.asn_from_list` to define the basic association of files with the different files, the rule (stage 2 or 3) and the final name of the product return by the stage 3.

</div>

The function `create_asn` creates ASN files to combined the dithers of each detector in input and one ASN file for all the dithers and detectors in a final mosaic.

In [ ]:
def create_asn(folder,detectors,filter_name):
    asnlist = []
    for detec in detectors:
        asnfile = 'l3asn_{}.json'.format(detec)
        cal_files = glob(os.path.join(folder, '*{}_cal.fits'.format(detec)))
        if cal_files != []:
            print('Step 3: Found ' + str(len(cal_files)) + ' input files to process for detector {}'.format(detec))
            # Name of the final product for the detector
            prodname = 'ima_{}_{}'.format(filter_name,detec)
            asnlist.append(asnfile)
            # Write the ASN file
            writeasn(cal_files,asnfile,prodname)

    # A common ASN file for all dithers and detectors
    asnfile = 'l3asn.json'
    cal_files = glob(os.path.join(folder, '*cal.fits'))
    prodname = 'ima_{}'.format(filter_name)
    writeasn(cal_files,asnfile,prodname)
    asnlist.append(asnfile)

    return(asnlist)

In [ ]:
# Useful function to write out a Lvl3 association file from an input list
def writeasn(files,asnfile,prodname):
    # Define the basic association of science files
    asn = jwst.associations.asn_from_list.asn_from_list(files,rule=DMS_Level3_Base,product_name=prodname)
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)

Function to run the stage 3 with the options we want
<div class="alert alert-block alert-info">
    
Here, we turn off the `tweakreg` option. With the `tweakreg` option turned off, the first two images are aligned, then the third one is aligned with the previous combination, etc. This allows for a better alignment between combined images. Alternatively, if the `tweakreg` option is on, during the alignment, all images are aligned with the first one, but since there is very little overlap between the first and the last dither, alignment is poor.

</div>

In [ ]:
def runimg3(filename, output_dir):
    # Instantiate the pipeline
    img3 = Image3Pipeline()
    # Specify where the output should go
    img3.output_dir = output_dir
    # Save the final resulting _rate.fits files
    img3.save_results = True
    # Skip the tweakreg
    img3.tweakreg.skip = True
    # Run the pipeline on an input list of files
    img3(filename)

In [ ]:
# Create the ASN files
#asnlist = create_asn(path_stage2, ['b1','b2','b3','b4'], 'f140m')
asnlist = create_asn(path_stage2, ['b1'], 'f140m')
print(asnlist)

In [ ]:
# Create a folder to store stage 3 products
path_stage3 = path_stage0.replace('stage0', 'stage3')
if not os.path.exists(path_stage3):
    os.makedirs(path_stage3)

In [ ]:
# Run the stage 3 for detector B1
runimg3('l3asn_b1.json',path_stage3)

Plot the final product for the detector B1

In [ ]:
b1_data = fits.getdata(os.path.join(path_stage3, 'ima_f140m_b1_i2d.fits'))
show(b1_data, 'Mosaic on detector b1', 'MJy/sr', max = 300)

Plot the final product

## Pipeline for Long wavelength

In this section, we will run the pipeline on the observations with the filter F335M, corresponding to long wavelengths. For long wavelengths, the used detectors are A5 and B5 but in our simulated data, we only have the observations for the module B.

### calwebb_detector1: Stage 1 Detector Processing

**Description**

This stage is the first one for all of the instruments. It corrects a part of instrument signatures in particu- lar by removing the readout pattern (the stripes). It is called`CALWEBB DETECTOR1` or `Detector1Pipeline` in the Python pipeline. The output of MIRAGE can be used here if during the simulation, datatype is set to raw. This stage only takes one file at a time and returns uncalibrated slope images in units of ADU/sec.

**Inputs**

* A raw exposure (`*_uncal.fits`) containing the 4-dimensional raw data from all detector readouts: (nintegrations x ngroups x ncols x nrows)


**Outputs**

* A 3D countrate image (`*_rateints.fits`) containing the results of each integration
* A 2D countrate image (`*_rate.fits`) corresponding of the average over the exposure's integrations

If there is only one integrate, `*_rateints.fits` and `*_rate.fits` are the same.

Data are not provided for this section. You can either create them with MIRAGE or look for them on the PDRs4All webpage.

In [ ]:
# Path containing the raw data
path_stage0 = './path/to/the/longwavelength/data/'
# Create a folder to store stage 1 products
path_stage1 = path_stage0.replace('stage0', 'stage1')
if not os.path.exists(path_stage1):
    os.makedirs(path_stage1)

In [ ]:
# Raw data
uncal_files = glob(os.path.join(path_stage0, '*uncal.fits'))

In [ ]:
# Run the stage 1 on all the raw data
for f in uncal_files:
    rundet1(f,path_stage1)

Plot the result of the rate file for the detector B5 and one dither

In [ ]:
rate_data = fits.getdata(os.path.join(path_stage1, 'jw01288001001_01101_00005_nrcb5_rate.fits'))

In [ ]:
show(rate_data,'Rate data', 'ADU/sec', max = 2500)

### calwebb_image2: Stage 2 Imaging Processing

**Description**

This stage corrects other instrumental signatures and calibrates the exposures. This step is performed using the `CALWEBB IMAGE` or `Image2Pipeline` in the Pipeline. This stage returns calibrated but unrectified slope images in units of MJy/sr.

**Inputs**

* A 2D countrate image (`*_rate.fits`) corresponding of the average over the exposure's integrations


**Outputs**

* A 2D calibrated, but unrectified, exposure (`*_cal.fits`)
* A 2D resampled, or rectified, image (`*_i2d.fits`)

The `*_i2d.fits` should not be used for science case because it did not pass through stage 3. At this stage, it can be used only for quick-look.

In [ ]:
# Rate data
rate_files = glob(os.path.join(path_stage1, '*rate.fits'))

# Create a folder to store stage 2 products
path_stage2 = path_stage0.replace('stage0', 'stage2')
if not os.path.exists(path_stage2):
    os.makedirs(path_stage2)

In [ ]:
# Run the stage 2 on all the rate data
for f in rate_files:
    runimg2(f,path_stage2)

Plot the result of the calibrated file for the detector B5 and one dither

In [ ]:
cal_data = fits.getdata(os.path.join(path_stage2, 'jw01288001001_01101_00005_nrcb5_cal.fits'))

In [ ]:
show(cal_data, 'Calibrated data', 'MJy/sr', max = 2500)

### calwebb_image3: Stage 3 Imaging Processing

**Description**

This stage combines multiple exposures from dithers or mosaics and rectifies the exposures to produce one unique mosaic, the final product. In this stage, we use the imaging mode called `CALWEBB IMAGE3` or `Image3Pipeline` in the Pipeline. To combine multiple exposures, an association (ASN) file is created which contains the images to be combined. Any combination of detectors and dithers is possible. This returns the final mosaic image, rectified, in units of MJy/sr.

**Inputs**

* A 2D calibrated, but unrectified, exposures (`*_cal.fits`) combined in an ASN file


**Outputs**

* A 2D resampled and combined image (`*_i2d.fits`) containing the combined and rectified association of exposures
* A 2D cosmic-ray flagged exposure (`*_crf.fits`) if `outlier_step == True`
* A source catalog (`*_cat.ecsv`) saved in ASCII
* A 2D segmentation map (`*_segm.fits`)

In [ ]:
# Create the ASN files
asnlist = create_asn(path_stage2, ['b5'], 'f335m')

In [ ]:
# Create a folder to store stage 3 products
path_stage3 = path_stage0.replace('stage0', 'stage3')
if not os.path.exists(path_stage3):
    os.makedirs(path_stage3)

In [ ]:
# Run the stage 3 for detector B5
runimg3('l3asn_b5.json',path_stage3)

Plot the final product for the detector B5

In [ ]:
b5_data = fits.getdata(os.path.join(path_stage3, 'ima_f335m_b5_i2d.fits'))
show(b5_data, 'Mosaic on detector b5', 'MJy/sr', max = 3000)